## 5장: 레이블이 없는 데이터를 활용한 사전 훈련

In [1]:
from importlib.metadata import version

pkgs = ["matplotlib",
        "numpy",
        "tiktoken",
        "torch",
        "tensorflow"]

for p in pkgs:
    print(f"{p} 버전: {version(p)}")

matplotlib 버전: 3.10.7
numpy 버전: 1.26.4
tiktoken 버전: 0.11.0
torch 버전: 2.6.0
tensorflow 버전: 2.20.0


### 5.1.1 GPT를 사용해 텍스트 생성하기

In [2]:
import torch
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 어휘 사전 크기
    "context_length": 256, # 짧은 문맥 길이 (원본 길이: 1024)
    "emb_dim": 768,        # 임베딩 차원
    "n_heads": 12,         # 어텐션 헤드 개수
    "n_layers": 12,        # 층 개수
    "drop_rate": 0.1,      # 드롭아웃 비율
    "qkv_bias": False      # 쿼리-키-값 생성시 편향 사용 여부
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

In [3]:
import tiktoken
from previous_chapters import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # 배치 차원을 삭제합니다.
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

출력 텍스트:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


### 5.1.2 텍스트 생성 손실 계산하기
- 두 개의 훈련 샘플(행)에 대한 토큰 ID를 담고 있는 inputs 텐서가 있다고 가정해 보죠
- inputs에 해당하는 targets은 모델이 생성해야 될 토큰 ID를 담고 있습니다.
- 2장에서 데이터 로더를 구현할 때 설명했듯이 targets은 inputs에서 한 토큰씩 앞으로 이동한 것입니다.

In [4]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

- inputs을 모델에 주입하면 각각 세 개의 토큰으로 구성된 두 개의 입력 샘플에 대한 로짓 벡터를 얻습니다.
- 각각의 토큰은 어휘 사전 크기에 해당하는 50,257 차원의 벡터입니다.
- 소프트맥스 함수를 적용하여 로짓 텐서를 확률 점수를 담고 있는 동일 차원의 텐서로 바꿀 수 있습니다.

In [5]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # 어휘 사전의 각 토큰에 대한 확률
print(probas.shape) # 크기: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


- 이전 장에서 설명했듯이 argmax함수를 적용하여 확률 점수를 토큰 ID (인덱스)로 바꿀 수 있습니다.
- 앞의 소프트맥스 함수는 각 토큰에 대해서 50,257차원의 벡터를 생성합니다. argmax 함수는 이 벡터에서 가장 높은 확률을 가진 위치를 반환합니다. 이것이 주어진 토큰에 대한 예측 토큰의 아이디입니다.
- 배치에는 각각 세개의 토큰으로 구성된 두 개의 입력 샘플이 있으므로 2x3크기의 예측 토큰을 얻습니다.

In [6]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("토큰 ID:\n", token_ids)

토큰 ID:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


- 이 토큰을 디코딩하면 모델이 예측해야 할 토큰, 즉 타겟 토큰과 매우 다른 것을 알 수 있습니다.

In [7]:
print(f"첫 번째 샘플의 타깃: {token_ids_to_text(targets[0], tokenizer)}")
print(f"첫 번째 샘플의 출력: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

첫 번째 샘플의 타깃:  effort moves you
첫 번째 샘플의 출력:  Armed heNetflix


- 타깃 인덱스에 해당하는 토큰 확률은 다음과 같습니다

In [8]:
targets[0]

tensor([3626, 6100,  345])

- 각 입력 샘플 토큰에 대한 정답 위치의 확률을 확인합니다

In [9]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("텍스트 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("텍스트 2:", target_probas_2)

텍스트 1: tensor([7.4540e-05, 3.1061e-05, 1.1563e-05])
텍스트 2: tensor([1.0337e-05, 5.6776e-05, 4.7559e-06])


- 확률이 1에 가까워지도록 이 값들을 최대화하는 것이 목표입니다.
- 수학적 최적화에서는 확률 점수 자체를 최대화하는 것보다 확률 점수의 로그를 최대화하는 것이 쉽습니다.

In [10]:
# 토큰 확률의 로그를 계산합니다.
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([ -9.5042, -10.3796, -11.3677, -11.4798,  -9.7764, -12.2561])


In [11]:
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

tensor(-10.7940)


- 모델 가중치를 최적화하여 평균 로그 확률을 가능한 크게 만드는 것이 목표입니다.
- 로그때문에 가장 큰 가능한 값은 0이며, 현재는 0에서부터 멀리 떨어져 있습니다.

- 딥러닝에서는 평균 로그 확률을 최대화하는 것 대신에 음의 평균 로그 확률을 최소화하는 것이 일반적입니다. 이 예제의 경우 -10.7940를 최대화하여 0에 가깝게 만드는 것 대신에 10.7940을 최소화하여 0에 가깝게 만듭니다.
- -10.7940의 음수 값, 즉, 10.7940을 딥러닝에서는 크로스 엔트로피 손실이라고 부릅니다

In [12]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(10.7940)


- cross_entropy 함수를 적용하기 전에 로짓과 타깃의 크기를 확인해야 합니다

In [13]:
# 로짓의 크기는 (batch_size, num_tokens, vocab_size) 입니다.
print("logits size: ", logits.shape)

# 타깃의 크기의 (batch_size, num_tokens) 입니다.
print("target size: ", targets.shape)

logits size:  torch.Size([2, 3, 50257])
target size:  torch.Size([2, 3])


- 파이토치의 cross_entropy함수를 위해 배치 차원을 기준으로 합쳐서 텐서를 펼쳐야 합니다.

In [14]:
# (batch, num_token, vocab_size) -> (total_tokens, vocab_size) vocab_size는 예측 벡터
# 모든 샘플, 모든 토큰 위치에 대한 예측을 하나의 큰 목록으로 간주, 각 행은 vocab_size 크기의 예측 벡터 
logits_flat = logits.flatten(0, 1) 
# targets는 (batch, num_tokens) 에는 vocab 내의 정답 인덱스
targets_flat = targets.flatten() # total_tokens 갯수의 정답 인덱스

print("flat logits: ", logits_flat.shape)
print("flat targets: ", targets_flat.shape)

flat logits:  torch.Size([6, 50257])
flat targets:  torch.Size([6])


In [15]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(10.7940)


### 5.1.3 훈련 세트와 검증 세트의 손실 계산하기

In [16]:
import os
import urllib.request

file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode("utf-8")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()

- 다운로드한 텍스트를 확인하기 위해 처음과 끝에서 100개의 문자를 출력

In [17]:
print(text_data[:99])
print(text_data[-99:])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."


In [18]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))

print("characters: ", total_characters)
print("tokens: ", total_tokens)

characters:  20479
tokens:  5145


In [19]:
from previous_chapters import create_dataloader_v1

# 훈련 세트 비율
train_ratio = 0.9
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

In [20]:
# 유효성 검사: context_length: 256
if total_tokens * (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("훈련 데이터 로더에 토큰이 충분하지 않습니다. "
          "`GPT_CONFG_124M['contxt_length']`를 낮추거나 "
          "`train_ratio`를 증가시키세요")
    
if total_tokens * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("훈련 데이터 로더에 토큰이 충분하지 않습니다. "
        "`GPT_CONFIG_124M['context_length']`를 낮추거나 "
        "`training_ratio`를 증가시키세요.")

- 데이터가 올바르게 로드되었는 지 확인

In [21]:
print("훈련 데이터 로더:")
for x , y in train_loader:
    print(x.shape, y.shape)
    
print("\n검증 데이터 로더:")
for x , y in val_loader:
    print(x.shape, y.shape)

훈련 데이터 로더:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

검증 데이터 로더:
torch.Size([2, 256]) torch.Size([2, 256])


- 토큰 크기가 예상 범위 안에 있는 지 추가로 확인합니다.

In [22]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()
    
val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()
    
print("훈련 토큰 수:", train_tokens)
print("검증 토큰 수:", val_tokens)
print("모든 토큰 수:", train_tokens + val_tokens)

훈련 토큰 수: 4608
검증 토큰 수: 512
모든 토큰 수: 5120


- 주어진 배치에서 크로스 엔트로피 손실을 계산하는 유틸리티 함수를 작성합니다.
- 또한 데이터 로더에서 사용자가 지정한 배치 개수 만큼 추출하여 손실을 계산하는 두 번째 유틸리티 함수를 구현합니다.

In [ ]:
def cal_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss

In [24]:
def cal_loss_loader(data_loader, model, device, num_batches=None):
    toatl_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)  # num_batch가 지정되지 않으면 모든 배치를 순회합니다.
    else:
        # num_batches가 데이터 로더에 있는 배치 개수보다 크면
        # 배치 횟수를 데이터 로더에 있는 총 배치 개수로 맟춥니다.
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = cal_loss_batch(input_batch, target_batch, model, device)
            toatl_loss += loss
        else:
            break
    return toatl_loss / num_batches

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with torch.no_grad():
    train_loss = cal_loss_loader(train_loader, model, device)
    val_loss = cal_loss_loader(val_loader, model, device)
print("훈련 손실:", train_loss)
print("검증 손실:", val_loss)

AttributeError: 'Tensor' object has no attribute 'flattern'